In [ ]:
import nltk

# Algoritmo di TextTiling formato da tre parti
## 1. Tokenization
Divide il testo in **unità lessicali**. I token sono tutti trasformati in caratteri minuscoli e altrimenti, ridotti alla loro radice (se non sono stop-words e vengono rimosse). Il testo viene suddiviso in pseudo-sequenze di dimensione *w*, dette **sequenze token**.
Le stop-words contribuiscono a comparare la dimensione della sequenza di token, ma non alla similarità tra blocchi.

## 2. Calcolo Lexical Score
Posso comparare la similarità di due blocchi guardando quante parole hanno in comune. Oppure posso usare un altro metodo dove assegno uno score di token-sequence gap in base a quante nuove parole trovo nell'intervallo a metà
### 2a. Blocchi
Raggruppo *k* token-sequences in un blocco, questo valore dovrebbe rappresentare la lunghezza media di un paragrafo.

## 3. Identificazione confini
Pre fare questo task assegno un depth score a ogni  gap di sequenze di token. Lo score corrisponde a quanto forte è il cambiamento dei blocchi di un gap, misurando la distanza da entrambe le punte al punto di minimo. E.g. depth score $a_1 = (y_{a1} - y_{a2}) + (y_{a3} - y_{a2})$